In [ ]:
# Copyright © 2021 LEADERG Inc. All rights reserved. Please keep it private. Publish to internet is not allowed. 

In [ ]:
import pandas as pd
import numpy as np
import sys

In [ ]:
# parameters
num1 = 280
num2 = 5

In [ ]:
# read csv
data = pd.read_csv("data/stock/stock-data.csv")

In [ ]:
# drop the data with volume = 0
for i in range(len(data)):
    if (data.volume[i] == '0'):
        data = data.drop(i)

In [ ]:
processed_data = dict()
for i in data.columns:
    if i == "date":
        processed_data["date"] = data.date
    else:
        dataNoNa = data[data.volume!='0']
        dataNoNa[i] = dataNoNa[i].replace(',', '', regex=True)
        dataNumeric = pd.to_numeric(dataNoNa[i])
        processed_data[i] = dataNumeric

In [ ]:
temp = pd.DataFrame()
for i in data.columns:
    temp = pd.concat([temp, pd.DataFrame(processed_data[i])], axis=1)
processed_data = temp

data_org = processed_data.copy()

temp = pd.DataFrame()

In [ ]:
# input data augmentation
for i in range(num1-1):
    day = i+1
    temp = data_org.loc[day:, ['volume', 'open', 'high', 'low', 'close']]
    temp = temp.reset_index(drop=True)
    processed_data = pd.concat([processed_data, temp], axis=1)

In [ ]:
# eliminate na
if num1 > 1:
    processed_data = processed_data[0:-(num1-1)]

date_tmp = data_org['date'].iloc[(num1-1):]
date_tmp = date_tmp.reset_index(drop=True)
processed_data['date'] = date_tmp
data_org_2 = data_org[(num1-1):]
data_org_2 = data_org_2.reset_index(drop=True)

In [ ]:
for i in range(num2):
    day = i+1
    data_tmp = processed_data.copy()
    temp = data_org_2['close'].iloc[day:]
    temp = temp.reset_index(drop=True)
    data_train = data_tmp[0:-day]
    
    if (day == 1):
        predicted_colume = 'close after ' + str(day) + ' day'
    else:
        predicted_colume = 'close after ' + str(day) + ' days'
    temp.name = predicted_colume
    #outputPath = 'data/stock-' + str(num1*5) + 'to' + str(num2) + '-' + str(day) + '.csv'
	#outputPath = 'data/stock-' + str(num1*5) + 'to' + str(num2) + '-' + str(day) + '.csv'
    #outputPath_validation = 'data/stock-' + str(num1*5) + 'to' + str(num2) + '-' + str(day) + '-validation.csv'
    

        
    # concat new columns
    data_train = pd.concat([data_train, temp], axis=1)
    data_inference = pd.concat([data_tmp, temp], axis=1)
    #print('temp[-1] =',temp[-1])
    data_inference[predicted_colume] = data_inference[predicted_colume].fillna(data_org['close'].iloc[-1])
    # validation proportion
    #validation = int(data_tmp.shape[0]*num3)
    #print('<', day, '> data_tmp.shape[0] = ', data_tmp.shape[0])
    #print('validation = ', validation)
    
    
    # save to csv
    outputPath = 'data/stock/train_input.csv'
    data_train.to_csv(outputPath, index=False)
    outputPath = 'data/stock/inference_input.csv'
    data_inference.to_csv(outputPath, index=False)
	
    #if validation > 0:
    #    data_tmp[0:-validation].to_csv(outputPath, index=False)
    #    data_tmp[-validation:].to_csv(outputPath_validation, index=False)
    #else:
    #    data_tmp.to_csv(outputPath, index=False)

print('準備完成')